In [141]:
ARTICLES_MIN_LENGTH = 300
ARTICLES_PER_AUTHOR = 250
LESS_ARTICLES_PER_AUTHOR = 100
LEAST_ARTICLES_PER_AUTHOR = 50
AUTHORS_TO_KEEP = 20

In [142]:
import pandas as pd
import matplotlib.pyplot as plt
import sqlite3

#https://ofai.github.io/million-post-corpus/
con = sqlite3.connect("../../../OneMillion/million_post_corpus/corpus.sqlite3", isolation_level=None,
                       detect_types=sqlite3.PARSE_COLNAMES)

df = pd.read_sql_query("SELECT distinct ID_User, COALESCE(Headline, '') || COALESCE(Body, '') as Beitrag from Posts", con)

url = r"https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()!@:%_\+.~#?&\/\/=]*)"

df = df.replace(to_replace=[r"\\t|\\n|\\r", "\t|\n|\r", url], value=[" "," ", "LINK"], regex=True)

df_minLength = df.loc[df['Beitrag'].apply(len) > ARTICLES_MIN_LENGTH]

In [143]:
## neuer Datensatz ohne User mit weniger als 250 Beiträgen
df_minArticles = df_minLength.groupby('ID_User').filter(lambda x : len(x) >= 250)

In [144]:
df_minArticles

,ID_User,Beitrag
592,28526,Im großen war es sehr beeindrucktend. Man kann...
605,28526,Mir bleibt ein sehr junger Mann (etwa so um di...
787,6498,ich habe die 1980er damit verbracht in den Ost...
1121,6247,"Vor etwas mehr vielen vielen Jahren, so gegen ..."
1240,6247,Ein Sternehotel im Montafon.In den späten 80ig...
...,...,...
1009952,29722,"Hier wird etwas getan, was bei Gender auch ger..."
1009996,29722,"Andersdenkende einfach als ""Pigs"" also Schwein..."
1009998,29722,Das ist eine lupenreine Verschwörungstheorie: ...
1010126,29722,"Sie machen jetzt genau das, was ich im unteren..."


In [145]:
## ID_User - Anzahl passende Beiträge
print(df_minArticles['ID_User'].value_counts())

5854     731
25987    613
22876    595
30344    574
14587    512
26866    473
28816    457
20141    419
24984    417
20349    409
3788     406
18083    401
1795     399
15686    382
18446    375
24667    374
6247     367
19915    366
8765     363
15886    360
10627    352
16722    342
3105     338
6498     336
24758    333
11656    332
27393    331
12471    328
30661    328
29722    318
26532    312
2690     304
1330     300
18940    300
18984    299
21150    299
11582    297
24343    294
12887    289
11910    286
29377    285
8446     285
25582    282
12071    281
12023    275
8323     272
1560     270
7439     266
6169     264
27991    264
16328    259
28526    256
18006    254
28084    251
Name: ID_User, dtype: int64


In [124]:
#df_minArticles['Text_length'] = df_minArticles['Beitrag'].apply(len)

In [146]:
df_minArticles

,ID_User,Beitrag
592,28526,Im großen war es sehr beeindrucktend. Man kann...
605,28526,Mir bleibt ein sehr junger Mann (etwa so um di...
787,6498,ich habe die 1980er damit verbracht in den Ost...
1121,6247,"Vor etwas mehr vielen vielen Jahren, so gegen ..."
1240,6247,Ein Sternehotel im Montafon.In den späten 80ig...
...,...,...
1009952,29722,"Hier wird etwas getan, was bei Gender auch ger..."
1009996,29722,"Andersdenkende einfach als ""Pigs"" also Schwein..."
1009998,29722,Das ist eine lupenreine Verschwörungstheorie: ...
1010126,29722,"Sie machen jetzt genau das, was ich im unteren..."


In [147]:
UserIDListe = df_minArticles.drop_duplicates(subset = ["ID_User"])['ID_User'].tolist()
print(len(UserIDListe))
print(UserIDListe)

54
[28526, 6498, 6247, 28816, 18083, 26866, 22876, 27393, 8765, 11656, 30661, 29377, 15886, 5854, 30344, 18446, 8446, 26532, 12887, 18006, 12071, 24984, 6169, 28084, 8323, 14587, 12023, 16328, 1560, 25987, 20141, 18940, 21150, 27991, 11582, 18984, 11910, 1795, 16722, 20349, 29722, 3105, 24758, 24343, 10627, 19915, 3788, 2690, 7439, 25582, 24667, 15686, 12471, 1330]


In [148]:
#20 Autoren die den Kriterien entsprechen:
IDS = UserIDListe[0:AUTHORS_TO_KEEP]
print(IDS)

[28526, 6498, 6247, 28816, 18083, 26866, 22876, 27393, 8765, 11656, 30661, 29377, 15886, 5854, 30344, 18446, 8446, 26532, 12887, 18006]


In [149]:
#250 Beiträge pro Autor werden zufällig gewählt
df_nArticlesPerUser = pd.DataFrame({"ID_User": [], 'Beitrag': []})
df_nArticlesPerUser['ID_User'] = df_nArticlesPerUser['ID_User'].astype(int)

for userID in IDS:
    df_nArticlesPerUser = pd.concat([df_nArticlesPerUser, df_minArticles.query("ID_User == @userID")[['ID_User','Beitrag']].sample(n = ARTICLES_PER_AUTHOR, replace = False)])
    
df_nArticlesPerUser = df_nArticlesPerUser.reset_index(drop=True)

In [150]:
df_nArticlesPerUser

,ID_User,Beitrag
0,28526,Man kann es auch mal umgekehrt betrachten. Sch...
1,28526,Die US-Primaries sind jedenfalls für mich der ...
2,28526,vielleicht bin ich da noch etwas zu traditione...
3,28526,Interessant ist allerdings das er offenbar die...
4,28526,Schweden ist möglicherweise das merkwürdigste ...
...,...,...
4995,18006,"""Sie versuchte aber auch einen ursprünglichen ..."
4996,18006,Wehrpflicht ist schon ein guter Ansatz :-)IÜ s...
4997,18006,"Trotz der geplatzten IT-Blase, die Bush von Cl..."
4998,18006,Griechenland hat schon sehr üble Tricks um die...


In [153]:
df_nArticlesPerUser.to_pickle("./250_posts_data.pkl")

In [155]:
#100 Beiträge pro Autor (aus den 250) werden zufällig gewählt
df_lessArticlesPerUser = pd.DataFrame({"ID_User": [], 'Beitrag': []})
df_lessArticlesPerUser['ID_User'] = df_lessArticlesPerUser['ID_User'].astype(int)

for userID in IDS:
    df_lessArticlesPerUser = pd.concat([df_lessArticlesPerUser, df_nArticlesPerUser.query("ID_User == @userID")[['ID_User','Beitrag']].sample(n = LESS_ARTICLES_PER_AUTHOR, replace = False)])
    
df_lessArticlesPerUser = df_lessArticlesPerUser.reset_index(drop=True)

In [156]:
df_lessArticlesPerUser

,ID_User,Beitrag
0,28526,Der öst. Arbeitslose wirkt irgendwie nachvollz...
1,28526,Wesentlicher und ehrlicher ist immer die absti...
2,28526,Eigentlich habe ich aber auch keine Lust mehr ...
3,28526,Bis zum 2. weltkrieg war Eugenik Mehrheitsmein...
4,28526,Ein Staatstreich ist per Definition ein Umstur...
...,...,...
1995,18006,Die politische Situation Ende der 1970er Jahre...
1996,18006,"Aber nein, China hat Apple Konditionen geboten..."
1997,18006,"Nein, der UN-Report erwähnt Russland, ebenso w..."
1998,18006,Zwar brachte die Ungarnkrise innerhalb kurzer ...


In [157]:
df_lessArticlesPerUser.to_pickle("./100_posts_data.pkl")

In [158]:
#50 Beiträge pro Autor (aus den 100) werden zufällig gewählt
df_leastArticlesPerUser = pd.DataFrame({"ID_User": [], 'Beitrag': []})
df_leastArticlesPerUser['ID_User'] = df_leastArticlesPerUser['ID_User'].astype(int)

for userID in IDS:
    df_leastArticlesPerUser = pd.concat([df_leastArticlesPerUser, df_lessArticlesPerUser.query("ID_User == @userID")[['ID_User','Beitrag']].sample(n = LEAST_ARTICLES_PER_AUTHOR, replace = False)])
    
df_leastArticlesPerUser = df_leastArticlesPerUser.reset_index(drop=True)

In [159]:
df_leastArticlesPerUser

,ID_User,Beitrag
0,28526,vielleicht bin ich da noch etwas zu traditione...
1,28526,also bis zum margaretenplatz finde ich ihn seh...
2,28526,"Das waren schon tolle Romane. Seinerzeit war ""..."
3,28526,Jedenfalls ein interessanter EInblick. Ich emp...
4,28526,Die jetzt kommenden werden zuallererst mal von...
...,...,...
995,18006,"Nein, der UN-Report erwähnt Russland, ebenso w..."
996,18006,Wie schon mehrmals hier bemerkt sind die Abges...
997,18006,Assad hat andere Probleme als Palmyra.Eine gro...
998,18006,"Die Angst, dass Flüchtlinge in Zelten schlafen..."


In [160]:
df_leastArticlesPerUser.to_pickle("./50_posts_data.pkl")